## 4. Donchian_Strategies

Buy when today's high > high of the past 40 days

Sell short when today's low < low of the past 40 days

Exit longs when today's low < low of the past 20 days

Exit shorts when today's high > high of the past 20 days

In [1]:
#coding=utf-8
# run_code_demo
from rqalpha import run_code

code = """
import rqalpha
from rqalpha.api import *
import talib
import numpy as np
import os

def init(context):
    context.s1 = "000001.XSHE"
    context.UPPERIOD = 40
    context.DOWNPERIOD = 20
    context.stoplossmultipler= 0.90 #止损 乘数
    context.takepofitmultipler= 4 #止盈 乘数


def handle_bar(context, bar_dict):
    stop_loss(context, bar_dict)
    prices = history_bars(context.s1, context.LONGPERIOD+1, '1d', 'close')
    volume = history_bars(context.s1, context.LONGPERIOD+1, '1d', 'volume')

    if len(prices)>context.LONGPERIOD:
        prices = history_bars(context.s1, context.UPPERIOD+2, '1d', 'close')
        high = history_bars(context.s1, context.UPPERIOD+2, '1d', 'high')
        low = history_bars(context.s1, context.DOWNPERIOD+2, '1d', 'low')

    if len(prices) > 41:
        upperband = ta.MAX(high, context.UPPERIOD)
        lowerband = ta.MIN(low, context.DOWNPERIOD)

        if bar_dict[context.s1].last < lowerband[-2] and prices[-2] > lowerband[-2] and cur_position > 0:
            order_shares(context.s1, 0)
        if bar_dict[context.s1].last > upperband[-2] and prices[-2] < upperband[-2] and cur_position == 0:
            order_target_value(context.s1, shares)

def stop_loss(context,bar_dict):
    for stock in context.portfolio.positions:
        if bar_dict[stock].last<context.portfolio.positions[stock].avg_price*context.stoplossmultipler:# 现价低于 原价一定比例
            order_target_percent(stock,0)
        elif bar_dict[stock].last>context.portfolio.positions[stock].avg_price*context.takepofitmultipler:# 现价高于原价一定比例
            order_target_percent(stock,0)

"""

config = {
  "base": {
    "start_date": "2000-06-01",
    "end_date": "2016-12-01",
    "securities": ['stock'],
    "stock_starting_cash": 1000000,
    "benchmark": "000300.XSHG"
#     "strategy_file_path": os.path.abspath(__file__)
  },
  "extra": {
    "log_level": "verbose",
  },
  "mod": {
    "sys_analyser": {
      "enabled": True,
      "plot": True
    }
  }
}

# 您可以指定您要传递的参数
run_code(code, config)